# Langchain + FAISS + Ollama

This workbook demonstrate RAG with FAISS & ollama (local)

**Notes:**
1. Istall ollama: https://ollama.com/download
2. Download ollama embed: `ollama pull nomic-embed-text`
3. Download the model (eg. `ollama pull mistral-openorca`)
4. `ollama serve`

In [0]:
import os
import glob

import gradio as gr

In [0]:
# imports for langchain

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document

from langchain.vectorstores import FAISS

from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

## Directory setup

In [0]:
# Read in documents using LangChain's loaders
# Take everything in all the sub-folders of our knowledgebase

folders = glob.glob("knowledge-base/*")

# With thanks to CG and Jon R, students on the course, for this fix needed for some users 
text_loader_kwargs = {'encoding': 'utf-8'}
# If that doesn't work, some Windows users might need to uncomment the next line instead
# text_loader_kwargs={'autodetect_encoding': True}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

In [0]:
# This step is to make sure setup above works
len(documents)

In [0]:
text_splitter = CharacterTextSplitter(chunk_size=1300, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

In [0]:
len(chunks)

In [0]:
doc_types = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Document types found: {', '.join(doc_types)}")

## Convert to vector database

In this version, I use `nomic-embed-text' from ollama library: https://ollama.com/library/nomic-embed-text.
Not sure if there are another embed model.

In [0]:
from langchain_ollama import OllamaEmbeddings, ChatOllama

db_name = "knowledge_base_1"
embeddings = OllamaEmbeddings(model="nomic-embed-text")

In [0]:
# Create our FAISS vectorstore!

vectorstore = FAISS.from_documents(chunks, embedding=embeddings)

total_vectors = vectorstore.index.ntotal
dimensions = vectorstore.index.d

print(f"There are {total_vectors} vectors with {dimensions:,} dimensions in the vector store")

## Chat setup

I add `callbacks=[StdOutCallbackHandler()]` parameter to see what exactly retriever passed to the model as Ed teach us.

In [0]:
# create a new Chat with Ollama
from langchain_core.callbacks import StdOutCallbackHandler

MODEL = "mistral-openorca"
llm = ChatOllama(temperature=0.7, model=MODEL)

# set up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# the retriever is an abstraction over the VectorStore that will be used during RAG
retriever = vectorstore.as_retriever(search_kwargs={"k": 13})

# putting it together: set up the conversation chain with the GPT 4o-mini LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, 
                                                           retriever=retriever, 
                                                           memory=memory, 
                                                           callbacks=[StdOutCallbackHandler()])

In [0]:
# run a quick test - should return a list of documents = 4
question = "Insurellm"
docs = vectorstore.similarity_search(question)
len(docs)

In [0]:
docs[0]

## Let's Chat!!

In [0]:
# Chat test
query = "tell me about Insurellm"
result = conversation_chain.invoke({"question": query})
answer = result["answer"]
print("\nAnswer:", answer)

In [0]:
# Clearing memory before new chat
memory.clear()

In [0]:
def chat_gradio(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]

In [0]:
# Ui launch
view = gr.ChatInterface(chat_gradio, type="messages").launch(inbrowser=True)